In [44]:
import re
import pandas as pd
from fuzzywuzzy import fuzz

# ===================================================
# Cleaning Tools
# ===================================================
counter1 = 0
counter2 = 0


def split_dataframe(df):
    """
    Splits a DataFrame with 4 columns into two DataFrames each containing
    sender name and amount.
    Assumes columns are ordered as: sender_name_1, amount_1, sender_name_2, amount_2.
    """
    try:
        df = df.dropna(axis=1, how="all")
        df = df.dropna(how="all")

        if len(df.columns) == 4 or len(df.columns) == 5:
            df = df.iloc[:, :4]
            df.columns = ["sender_name_1", "amount_1", "sender_name_2", "amount_2"]
            df1 = df[["sender_name_1", "amount_1"]].rename(
                columns={"sender_name_1": "sender_name", "amount_1": "amount"}
            )
            df2 = df[["sender_name_2", "amount_2"]].rename(
                columns={"sender_name_2": "sender_name", "amount_2": "amount"}
            )

        elif len(df.columns) == 6:
            df = df.iloc[:, :6]
            df.columns = [
                "code_type",
                "code_number",
                "sender_name_1",
                "amount_1",
                "sender_name_2",
                "amount_2",
            ]
            df1 = df[["code_type", "code_number", "sender_name_1", "amount_1"]].rename(
                columns={"sender_name_1": "sender_name", "amount_1": "amount"}
            )
            df2 = df[["sender_name_2", "amount_2"]].rename(
                columns={"sender_name_2": "sender_name", "amount_2": "amount"}
            )

    except Exception as e:
        print("The error is probably due to number of columns is neither 4/5/6")
        print("Error in split_dataframe: ", e)
        return None, None

    return df1.dropna(axis=0, how="all"), df2.dropna(axis=0, how="all")


def clean_dataframe(df):
    df["sender_name"] = df["sender_name"].fillna("").astype(str)
    df.amount = df.amount.abs()
    # Suppose df is your DataFrame
    df["sender_name"] = df["sender_name"].astype(str)
    df.sender_name = df.sender_name.str.strip().str.lower()
    return df


def extract_clean_name(text):
    """
    Cleans and tokenizes a sender name string by removing currency symbols,
    numbers, and splitting on punctuation or whitespace.
    """
    if not isinstance(text, str):
        return []

    text = re.sub(r"(R\$|\$|USD)?\s?\d+(\.\d{1,2})?(Buy@\d+(\.\d{1,2})?)?", "", text)
    tokens = re.split(r"[ -+()@,]+", text.lower())
    return [item for item in tokens if item.strip()]


# ===================================================
# Name Matching Tools
# ===================================================


def is_name_match_1(tokens1, tokens2):
    """
    Lightweight name match function using substring token comparison.
    """
    global counter1
    global counter2

    if len(tokens2)==1:
        t2 = tokens2[0]
        for t1 in tokens1:
            if len(t2) > 2 and t2 in t1:
                counter2 += 1
                return True
            else:
                pass
                # print("No match: ", t1, t2)
    
    elif len(tokens2) > 1:
        if all(t2 in tokens1[:2] for t2 in tokens2) and all(len(t2) > 1 for t2 in tokens2):
            counter2 += 1
            return True

    return False

def is_name_match_2(tokens1, tokens2):
    """
    Determines if two names are a fuzzy match using token-wise comparison
    and fuzzy ratios.
    """

    if not tokens1 or not tokens2:
        return False

    for t1 in tokens1:
        for t2 in tokens2:
            if (
                fuzz.ratio(t1, t2) >= 80  # Lower the threshold
                or fuzz.partial_ratio(t1, t2) >= 80  # Allow partial matches
            ) and (len(t1) > 2 and len(t2) > 2):
                return True

    joined1 = " ".join(tokens1)
    joined2 = " ".join(tokens2)
    return fuzz.token_set_ratio(joined1, joined2) >= 75


# ===================================================
# Matching Tools
# ===================================================


# layer 1 of matching
def find_unmatched_rows(df1, df2):
    """
    Compares two DataFrames and returns rows from both that have no matching
    (amount + name) in the other. Uses a lightweight name match.
    """
    unmatched_df1 = []
    unmatched_df2 = []
    df2["matched"] = False  # Ensure the column exists and is False by default

    for i1, row1 in df1.iterrows():
        matched = False
        for i2, row2 in df2.iterrows():
            if row1["amount"] == row2["amount"]:
                tokens1 = extract_clean_name(row1["sender_name"])
                tokens2 = extract_clean_name(row2["sender_name"])
                if is_name_match_1(tokens1, tokens2):
                    matched = True
                    df2.at[i2, "matched"] = True
                    break
        if not matched:
            unmatched_df1.append(row1)
        
    for i2, row2 in df2.iterrows():
        if not row2.get("matched", False):
            unmatched_df2.append(row2)

    return pd.DataFrame(unmatched_df1).reset_index(drop=True), pd.DataFrame(unmatched_df2).reset_index(drop=True)


# layer 2 of matching
def match_names_on_amount(df1, df2):
    """
    Matches rows from two DataFrames based on equal amounts and fuzzy name match.
    Returns matched pairs along with unmatched rows from both DataFrames.
    """
    matched_rows = []
    matched_indices_df1 = set()
    matched_indices_df2 = set()

    for i, row1 in df1.iterrows():
        amount1 = row1["amount"]
        name1 = row1["sender_name"]

        matching_df2 = df2[df2["amount"] == amount1]
        if not isinstance(name1, str):
            return False
        tokens1 = extract_clean_name(name1)

        for j, row2 in matching_df2.iterrows():
            name2 = row2["sender_name"]
            if not isinstance(name2, str):
                return False
            tokens2 = extract_clean_name(name2)

            if is_name_match_2(tokens1, tokens2):
                matched_rows.append(
                    {
                        "df1_index": i,
                        "df1_name": name1,
                        "df1_amount": amount1,
                        "df2_index": j,
                        "df2_name": name2,
                        "df2_amount": row2["amount"],
                    }
                )
                matched_indices_df1.add(i)
                matched_indices_df2.add(j)
                break  # one-to-one match

    matched_df = pd.DataFrame(matched_rows)
    unmatched_df1 = df1[~df1.index.isin(matched_indices_df1)].copy()
    unmatched_df2 = df2[~df2.index.isin(matched_indices_df2)].copy()

    return matched_df, unmatched_df1, unmatched_df2

In [45]:
# ===================================================
# Main Function
# ===================================================
"""
Main function to load, clean, and match transaction data from Excel file.
Returns:
    matched_df: Matched transactions between df1 and df2.
    unmatched_df1: Transactions from df1 with no match.
    unmatched_df2: Transactions from df2 with no match.
"""
# excel_path = r"../excel_data\table_test.xlsx"
excel_path = r"../excel_data\table.xlsx"
# excel_path = r'../'


file_type = excel_path.split(".")[-1]

if file_type == "xls":
    df = pd.read_excel(excel_path, engine="xlrd")
else:
    df = pd.read_excel(excel_path)

# Split and clean
df1, df2 = split_dataframe(df)
df1 = clean_dataframe(df1)
df2 = clean_dataframe(df2)

# Pre-filter unmatched using lightweight comparison
unmatched_df1, unmatched_df2 = find_unmatched_rows(df1, df2)

# Run full fuzzy matching
matched_df, unmatched_df1, unmatched_df2 = match_names_on_amount(
    unmatched_df1,
    unmatched_df2,
)

# Final cleanup
matched_df = matched_df.drop(columns=["df1_index", "df2_index"], errors="ignore")
matched_df1, matched_df2 = split_dataframe(matched_df)
if "matched" in unmatched_df2.columns:
    unmatched_df2 = unmatched_df2.drop("matched", axis=1)

print("counter1: ", counter1)
print("counter2: ", counter2)

print(matched_df1)
print()
print(matched_df2)
print()

counter1:  0
counter2:  237
                                     sender_name     amount
0                    csv promocao de vendas ltda  117454.36
1                         priscila franz kreusch     675.00
2                             johny dias scalcon     470.00
3                          nicolas silva de deus    5750.00
4                                      ali waked   11125.00
5                      globo das utilidades ltda    4000.00
6              h j e zein celular e eletronic...    9000.00
7                          josiele kenia barbosa    1023.00
8           mi amozanas comercio de equipamentos   10000.00
9                         drobenche imports ltda    3550.00
10                       wannesa da silva dantas    8250.00
11                   jhonatas dos santos barbosa    5529.00
12                a e s comercio e servicos ltda    4500.00
13                a e s comercio e servicos ltda    4500.00
14        ac eletronicos acessorios e pecas ltda    4000.00
15          

In [47]:
print(unmatched_df1)
print(unmatched_df1.shape)
print()
print(unmatched_df2)
print(unmatched_df2.shape)

                                sender_name     amount
7                    matheus henrique borin    7000.00
11                     princesas das fibras    1800.00
14                              a veronezzi     130.00
15                     iago lins cavalcanti    5000.00
18              marlene marobin 04721641982     772.00
19                tabacaria el patrono ltda    5900.00
20                     a t p funeraria ltda    1000.00
21                            mohamad awala   26000.00
22               francisco carlito silveira    2100.00
23                             ibrahim harb    5000.00
24                   santiago comercio ltda   11882.00
26                     k b silva dos santos    6000.00
28  macel imports / loja j m s imports ltda    1200.00
29                           r f dos santos   20003.95
31             aldonei coitinho de castilho    1830.00
32                                    kinxt     700.00
33                                     foul  118511.95
34        

In [30]:
df1

,sender_name,amount
0,matheus aparecido da silva oliveira,3320.00
1,lucas giehl carvalheiro,409.45
2,leandro phones ltda,2230.00
3,csv promocao de vendas ltda,117454.36
4,ms parts,1000.00
...,...,...
267,foul,118511.95
268,cohiba,14900.00
269,cohiba,16997.00
270,charuto sub1,177320.00


In [14]:
df.tail()

,TRK+DEBORA,-40,DEBORA,40
94,R$ 22919.40Buy@5.93 TRKBIT-MIXCELL (USD 38...,-22919.4,MIXCELL,22919.40
95,NaN,NaN,NaN,NaN
96,NaN,NaN,KINXT,35074.65
97,NaN,NaN,FOUL,145208.18
98,NaN,NaN,CHARUTO,167540.00
